In [1]:
import sys
sys.path.append("..")
from src.preprocess.data_prep_offenseval import DataPrepOffensEval
from src.classifiers.classifier_bi_lstm import BiLstmClassifier
from src.utils.stats import get_distribution_from_y
import pandas as pd

/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/local/Cellar/python/3.7.2_2/Frameworks/Python.framework/Versions/3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/usr/local/Cellar/python

In [2]:
ENGLISH_FILE_PATH = "../data/raw/OffensEval2019/start-kit/training-v1/offenseval-training-v1.tsv"
def get_X_y():
    dp = DataPrepOffensEval()
    result_tuple = dp.get_X_and_ys(file_path=ENGLISH_FILE_PATH)
    X_original = result_tuple[0]
    y_sub_a = result_tuple[1]
    return X_original, y_sub_a

In [3]:
# Format our text samples and labels into tensors that can be fed into 
# a NN. To do this, we will rely on Keras utilities
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_NB_WORDS = 20000 # Max number of words in tweet to consider, ranked by frequency
MAX_SEQ_LENGTH = 1000 # Lengt of tensors that are fed into NN

def tokenize(X, max_nb_words=MAX_NB_WORDS, max_seq_length=MAX_SEQ_LENGTH):
    tokenizer = Tokenizer(num_words=max_nb_words)
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)
    word_index = tokenizer.word_index # Number of unique tokens
    X = pad_sequences(sequences, maxlen=max_seq_length) # PAD Everything to be the same length - necessary for keras
    return X, word_index

Using TensorFlow backend.


In [4]:
X, y = get_X_y()

In [5]:
X, word_index = tokenize(X)

In [6]:
dp = DataPrepOffensEval()
X_train, X_test, y_train, y_test = dp.train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = dp.train_test_split(X_train, y_train, test_size=0.1)

In [8]:
from gensim.models import KeyedVectors
NUM_VECTORS = 50000
ENGLISH_EMB_FILE_PATH = "../cc.en.300.vec"

en_model = KeyedVectors.load_word2vec_format(
    ENGLISH_EMB_FILE_PATH,
    limit=NUM_VECTORS,
)

In [11]:
import numpy as np

def get_embedding_matrix(model=en_model, X=X, embedding_dim=300, word_index=word_index):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in model:
            embedding_vector = model[word]
            embedding_matrix[i] = embedding_vector
            # Words not found will be all zeros
    return embedding_matrix

In [12]:
embedding_matrix = get_embedding_matrix()

In [15]:
# Create the embedding layer from the embedding matrix for keras
from keras.layers import Embedding
EMBEDDING_DIM = 300

embedding_layer = Embedding(
    len(word_index) + 1,
    EMBEDDING_DIM,
    weights=[embedding_matrix],
    input_length=MAX_SEQ_LENGTH,
    trainable=False,
)

In [ ]:
from keras.layers import (
    Bidirectional, 
    Dropout, 
    Dense,
    Activation, 
    LSTM,
)

from keras import Sequential

def create_model(
    embedding_layer,
    dropout_amount=0.2,
    lstm_units=10,
    dense_1_units=4,
    dense_2_units=1,
    dense_1_activation="relu",
    dense_2_activation="sigmoid",
    optimizer="adam",
    loss_func="binary_crossentropy",
    metrics=["accuracy"],
):
    model = Sequential()
    model.add(embedding_layer)
    model.add(Bidirectional(
        layer=LSTM(
            units=lstm_units,
            dropout=dropout_amount,
            recurrent_dropout=dropout_amount,
        )
    ))
    model.add(Dropout(dropout_amount))
    model.add(Dense(
        units=dense_1_units,
    ))
    model.add(Activation(dense_1_activation))
    model.add(Dropout(dropout_amount))
    model.add(Dense(
        units=dense_2_units,
    ))
    model.add(Activation(dense_2_activation))
    
    model.compile(
        optimizer=optimizer,
        loss=loss_func,
        metrics=metrics,
    )
    
    return model

def train_model(
    model,
    X_train=X_train,
    y_train=y_train,
    epochs=20,
    batch_size=512,
    validation_data=[X_val,y_val],
):
    history = model.fit(
        X_train,
        y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=validation_data,
        verbose=2,
    )
    return history

import matplotlib.pyplot as plt


def plot_train_val_loss(history):
    plt.clf()
    history_dict = history.history
    train_loss = history_dict['loss']
    val_loss = history_dict['val_loss']
    epochs = range(1, len(train_loss) + 1)
    plt.plot(epochs, train_loss, 'r', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    
def plot_train_val_accuracy(history):
    plt.clf()
    history_dict = history.history
    train_metric = history_dict['acc']
    val_metric = history_dict['val_' + 'acc']
    epochs = range(1, len(train_metric) + 1)
    plt.plot(epochs, train_metric, 'r', label='Training {}'.format('acc'))
    plt.plot(epochs, val_metric, 'b', label='Validation {}'.format('acc'))
    plt.title('Training and Validation {}'.format('acc'))
    plt.xlabel('Epochs')
    plt.ylabel('{}'.format('acc'))
    plt.legend()

    
def predict_y(model, X_test=X_test):
    y_pred = model.predict(X_test)
    y_pred_binary = [0 if value[0] < 0.5 else 1 for value in y_pred]
    return y_pred_binary


# Using this since we have the confusion matrix stuff implemented here
# Move that stuff into a unique module
from src.classifiers.classifier_bi_lstm import BiLstmClassifier


def plot_confusion_matrix(
    y_test,
    y_pred,
    num_categories=2,
    category_names=["NOT", "OFF"],
):
    classifier = BiLstmClassifier(
        embedding_input_dim=1,
        logfile=None,
        epochs=1,
    )
    confusion_df = classifier.confusion_matrix(
        y_test, y_pred, num_categories, category_names,
    )
    classifier.plot_confusion_matrix(
        confusion_df, file_path=None,
    )


In [22]:
model = create_model(embedding_layer)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
history = train_model(model)

Instructions for updating:
Use tf.cast instead.
Train on 9532 samples, validate on 1060 samples
Epoch 1/20
 - 100s - loss: 0.6784 - acc: 0.6536 - val_loss: 0.6597 - val_acc: 0.6783
Epoch 2/20
 - 103s - loss: 0.6523 - acc: 0.6615 - val_loss: 0.6314 - val_acc: 0.6783
Epoch 3/20
 - 102s - loss: 0.6386 - acc: 0.6625 - val_loss: 0.6241 - val_acc: 0.6783
Epoch 4/20
 - 103s - loss: 0.6351 - acc: 0.6626 - val_loss: 0.6198 - val_acc: 0.6783
Epoch 5/20
 - 110s - loss: 0.6291 - acc: 0.6626 - val_loss: 0.6113 - val_acc: 0.6783
Epoch 6/20
